# Week4 .dti Dependency Confirmation

This notebook validates dependencies from `requirements.txt` in the Week4 environment.


In [1]:
import os
import platform
import sys

print('Executable:', sys.executable)
print('Python:', sys.version.replace('\n', ' '))
print('Platform:', platform.platform())
print('Working dir:', os.getcwd())


Executable: /Users/sneha/Downloads/Final Year/Week4/.dti/bin/python
Python: 3.11.14 (main, Oct  9 2025, 16:16:55) [Clang 17.0.0 (clang-1700.6.3.2)]
Platform: macOS-26.3-arm64-arm-64bit
Working dir: /Users/sneha/Downloads/Final Year/Week4


In [2]:
from pathlib import Path
import re
import pandas as pd

REQ_FILE = Path('requirements.txt')

def parse_requirement_line(line: str):
    line = line.strip()
    if not line or line.startswith('#'):
        return None
    if line.startswith('-r') or line.startswith('--requirement'):
        return None
    line = line.split(' #', 1)[0].strip()
    line = line.split(';', 1)[0].strip()
    match = re.match(r'^([A-Za-z0-9_.-]+)', line)
    if not match:
        return None
    return {'requirement': line, 'package': match.group(1)}

raw_lines = REQ_FILE.read_text(encoding='utf-8').splitlines()
records = [r for r in (parse_requirement_line(line) for line in raw_lines) if r is not None]
req_df = pd.DataFrame(records)

print(f'Requirement file: {REQ_FILE.resolve()}')
print(f'Parsed dependencies: {len(req_df)}')
req_df


Requirement file: /Users/sneha/Downloads/Final Year/Week4/requirements.txt
Parsed dependencies: 20


,requirement,package
0,numpy==1.26.4,numpy
1,pandas==2.2.1,pandas
2,scipy==1.12.0,scipy
3,matplotlib==3.8.3,matplotlib
4,seaborn==0.13.2,seaborn
5,scikit-learn==1.4.1.post1,scikit-learn
6,xgboost==2.0.3,xgboost
7,torch==2.2.1,torch
8,torchvision==0.17.1,torchvision
9,torchaudio==2.2.1,torchaudio


In [3]:
from importlib.metadata import PackageNotFoundError, version

rows = []
for rec in records:
    pkg = rec['package']
    try:
        installed_version = version(pkg)
        installed = True
    except PackageNotFoundError:
        installed_version = None
        installed = False

    rows.append({
        'package': pkg,
        'requirement': rec['requirement'],
        'installed': installed,
        'installed_version': installed_version,
    })

installed_df = pd.DataFrame(rows)
installed_df


,package,requirement,installed,installed_version
0,numpy,numpy==1.26.4,True,1.26.4
1,pandas,pandas==2.2.1,True,2.2.1
2,scipy,scipy==1.12.0,True,1.12.0
3,matplotlib,matplotlib==3.8.3,True,3.8.3
4,seaborn,seaborn==0.13.2,True,0.13.2
5,scikit-learn,scikit-learn==1.4.1.post1,True,1.4.1.post1
6,xgboost,xgboost==2.0.3,True,2.0.3
7,torch,torch==2.2.1,True,2.2.1
8,torchvision,torchvision==0.17.1,True,0.17.1
9,torchaudio,torchaudio==2.2.1,True,2.2.1


In [4]:
from packaging.requirements import Requirement

def satisfies(requirement_str: str, installed_version: str | None) -> bool:
    if installed_version is None:
        return False
    req = Requirement(requirement_str)
    if not req.specifier:
        return True
    return installed_version in req.specifier

check_df = installed_df.copy()
check_df['meets_requirement'] = [
    satisfies(req, ver) for req, ver in zip(check_df['requirement'], check_df['installed_version'])
]

missing = check_df[~check_df['installed']]
mismatch = check_df[check_df['installed'] & ~check_df['meets_requirement']]

print(f'Missing packages: {len(missing)}')
print(f'Installed but version-mismatched: {len(mismatch)}')
print(f"Fully satisfied: {int(check_df['meets_requirement'].sum())} / {len(check_df)}")

check_df


Missing packages: 0
Installed but version-mismatched: 0
Fully satisfied: 20 / 20


,package,requirement,installed,installed_version,meets_requirement
0,numpy,numpy==1.26.4,True,1.26.4,True
1,pandas,pandas==2.2.1,True,2.2.1,True
2,scipy,scipy==1.12.0,True,1.12.0,True
3,matplotlib,matplotlib==3.8.3,True,3.8.3,True
4,seaborn,seaborn==0.13.2,True,0.13.2,True
5,scikit-learn,scikit-learn==1.4.1.post1,True,1.4.1.post1,True
6,xgboost,xgboost==2.0.3,True,2.0.3,True
7,torch,torch==2.2.1,True,2.2.1,True
8,torchvision,torchvision==0.17.1,True,0.17.1,True
9,torchaudio,torchaudio==2.2.1,True,2.2.1,True


In [5]:
import subprocess
import sys

result = subprocess.run(
    [sys.executable, '-m', 'pip', 'check'],
    capture_output=True,
    text=True,
)

output = (result.stdout or result.stderr).strip()
print(output if output else 'pip check returned no output.')
print('pip check exit code:', result.returncode)


No broken requirements found.
pip check exit code: 0


## Result Guide
- `installed=False`: package is missing in the current environment.
- `meets_requirement=False`: package is installed but does not match the required version/specifier.
- `pip check exit code 0`: dependency graph is internally consistent.
